## Setup

In [ ]:
import getpass  # For input prompt not to show what is entered
import json     # Provide convinent functions to handle json objects 
import re       # For regular expression
import requests # To handle http requests

import warnings
warnings.simplefilter('ignore') # Because I am using unverified ssl certificates 

baseurl = 'https://demo.loris.ca/api/v0.0.2' # Pick yours

## Exercise 1. Login

This is a POST request to the `/login` endpoint that requires 2 parameters: `username` and `password`  
The expected response is a json string that contains a token property.  

https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#11-authentication

In [ ]:
payload = {
    'username': input('username: '), 
    'password': getpass.getpass('password: ')
}

response = requests.post(
    url = baseurl + '/login',
    json = payload,
    verify = False
)

text = response.content.decode('ascii')

data = json.loads(text)

print(data)

*Store the token in a variable for later*

In [ ]:
token = data['token']

## Exercise 2. Print all candidates candid

This is a GET request to /candidates  

https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#30-candidate-api

In [ ]:
response = json.loads(requests.get(
    url = baseurl + '/candidates',
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
).content.decode('ascii'))

In [ ]:
print(json.dumps(response, indent=2, sort_keys=True))

In [ ]:
candidates = response['Candidates']
for candidate in candidates:
    print(candidate['CandID'])

## Exercise 3. Find all candidates and session with a given instruments

This is a series of GET request  

https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#31-specific-candidate  
https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#33-candidate-instruments   
https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#33-the-candidate-instrument-data

In [ ]:
instrument = 'bmi'

for candidate in candidates:
    candid = candidate['CandID']
    response = json.loads(requests.get(
        url = baseurl + '/candidates/' + candid,
        verify = False,
        headers = {'Authorization': 'Bearer %s' % token}
    ).content.decode('ascii'))
    visit_labels = response['Visits']
    for visit_label in visit_labels:
        response = json.loads(requests.get(
            url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments',
            verify = False,
            headers = {'Authorization': 'Bearer %s' % token}
        ).content.decode('ascii'))
        if instrument in response['Instruments']:
            response = json.loads(requests.get(
                url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments/' + instrument,
                verify = False,
                headers = {'Authorization': 'Bearer %s' % token}
            ).content.decode('ascii'))
            print(json.dumps(response, indent=2, sort_keys=True))

## Exercise 4. Input instrument data for a candidate
  
PUT or PATCH request to /candidates/$CandID/$VisitLabel/instruments/$InstrumentName  

https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#33-the-candidate-instrument-data

data format:
```json
{
  "Meta": {
    "Candidate": string,
    "DDE": true|false,
    "Instrument": string,
    "Visit": string
  },
  <instrument_name>: {
    <field1_name>: <value1>,
    <field2_name>: <value2>,
    ...
  }
}
```


In [ ]:
def getInstrumentData(candid, visit_label, instrument):
    return json.loads(requests.get(
        url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments/' + instrument,
        verify = False,
        headers = {'Authorization': 'Bearer %s' % token}
    ).content.decode('ascii'))

def prettyPrint(string):
    print(json.dumps(string, indent=2, sort_keys=True))
    
instrument = 'bmi'
candid = '306843'
visit_label = 'V01'

prettyPrint(getInstrumentData(candid, visit_label, instrument))

#### PUT request containing all the fields

In [ ]:
# Get all the fields an meta data
json_input = getInstrumentData(candid, visit_label, instrument)

# Update one field
json_input[instrument]['weight_kgs'] = 3

r = requests.put(
    url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments/' + instrument,
    json = json_input,
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
)
print(r.status_code) # <Response [500]> demo.loris.ca do not handle that on LORIS 16...

prettyPrint(getInstrumentData(candid, visit_label, instrument))

#### PATCH request containing some of the fields

In [ ]:
# Get all the meta data
old_values = getInstrumentData(candid, visit_label, instrument)
new_values = {}
new_values['Meta'] = old_values['Meta']

# Add fields in the instrument object
new_values[instrument] = {}
new_values[instrument]['weight_kgs'] = 2
new_values[instrument]['height_inches'] = 5

prettyPrint(new_values)


In [ ]:
r = requests.patch(
    url = baseurl + '/candidates/' + candid + '/' + visit_label + '/instruments/' + instrument,
    json = new_values,
    verify = False,
    headers = {'Authorization': 'Bearer %s' % token}
)
print(r.status_code) # <Response [500]> demo.loris.ca do not handle that on LORIS 16...

prettyPrint(getInstrumentData(candid, visit_label, instrument))

## Exercise 5. Create a candidate and add a V01 session to it  

https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#30-candidate-api  
https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#32-getting-candidate-visit-data  


In [ ]:
json_data = {
    "Candidate" : {
        'Project' : 'loris',
        'Site'    : 'Montreal',
        'DoB'     : '2008-10-05',
        'EDC'     : '2008-10-05',
        'Gender'  : 'Female'
    }
}
response = json.loads(requests.post(
    url = baseurl + '/candidates/',
    verify = False,
    json = json_data,
    headers = {'Authorization': 'Bearer %s' % token}
).content.decode('ascii'))

print(response)

In [ ]:
# Add a Visit
candid = str(response['Meta']['CandID'])
site = 'Montreal'
visit_label = 'V01'
battery = 'Control'

json_data = {
    'Meta' : {
        'CandID'  : candid,
        'Visit'   : visit_label,
        'Site'    : site,
        'Battery' : battery
    }
}

r = requests.put(
    url = baseurl + '/candidates/' + candid + '/' + visit_label,
    verify = False,
    json = json_data,
    headers = {'Authorization': 'Bearer %s' % token}
)
print(r)